In [3]:
__all__ = ['NumericalSemigroup']  # So NumericalSemigroup is imported when "from <module> import *" is used
from collections import deque

class NumericalSemigroup:
    def __init__(self, generators):
        if not isinstance(generators, list):
            raise TypeError("Generators must be provided as a list.")
        self.gens = generators
        self.multiplicity = min(generators)
        self.gaps = self.compute_gaps()
        self.Frobenius = max(self.gaps) if self.gaps else None
        self.genus = len(self.gaps)

    def Apery_set(self, m):
        # Compute Apery set from generators
        # ONLY WORKS WHEN m IS A GENERATOR
        # returns Apery set as list
        gens = self.gens
        gensnoM = gens.copy()
        gensnoM.remove(m)

        C = (m-1)*(sum(gensnoM)) + 1
        
        Apery_set = [C] * m
        Apery_set[0] = 0
        
        Q = deque([0])

        while Q:
            n = Q.popleft()
            x = n % m
            if n <= Apery_set[x]:
                for g in gensnoM:
                    y = n + g
                    z = y % m
                    if y < Apery_set[z]:
                        Apery_set[z] = y
                        Q.append(y)
        Apery_set.remove(0)
        return Apery_set

    def compute_gaps(self):
        m = self.multiplicity
        Ap = self.Apery_set(m)
        gaps = []
        for i in Ap:
            while i > m:
                i = i - m
                gaps.append(i)
        gaps = sorted(set(gaps))
        return gaps
        
    def Kunz_Coords(self):
        m = self.multiplicity
        Ap = self.Apery_set(m)
        lAp = list(range(len(Ap)))
        Kunz = []
        for i in lAp:
            Kunz.append((Ap[i] - (i + 1)) / m)
        return Kunz
        
    def Remove_Min_Gen(self, r):
        # r is the mingen you want to remove
        Gens = self.gens.copy()
        Gens.remove(r)
        m = self.multiplicity
        
        if r == m:
            Gens.append(2 * r)
            Gens.append(2 * r + 1)
            return Gens
            
        k = m + r
        genslessr = [i for i in Gens if i < r]

        queue = deque(genslessr)
        while queue:
            n = queue.popleft()
            for g in genslessr:
                y = g + n
                if y < k:
                    queue.append(y)
                if y == k:
                    return Gens
        Gens.append(k)
        return Gens
        
    def Children(self):
        #takes numerical semigroup
        #returns its children as a list of numerical semigroups
        Gens = self.gens
        F = self.Frobenius
        C = []  # create list of children
        for i in Gens:
            if F < i:  # if i is effective gen
                newGens = self.Remove_Min_Gen(i)
                C.append(NumericalSemigroup(newGens))
        return C 

In [4]:
generators = [4, 6, 7, 9]
semigroup = NumericalSemigroup(generators)
print(f"Generators: {semigroup.gens}")
print(f"Apery set: {semigroup.Apery_set(4)}")
print(f"Gaps: {semigroup.compute_gaps()}")
print(f"Kunz_Coords: {semigroup.Kunz_Coords()}")
children = semigroup.Children()
print(f"Children: {[child.gens for child in children]}")
sgp = NumericalSemigroup([5, 6, 7, 8, 9])
print(sgp.Remove_Min_Gen(5))

Generators: [4, 6, 7, 9]
Apery set: [9, 6, 7]
Gaps: [1, 2, 3, 5]
Kunz_Coords: [2.0, 1.0, 1.0]
Children: [[4, 7, 9, 10], [4, 6, 9, 11], [4, 6, 7]]
[6, 7, 8, 9, 10, 11]


In [5]:
class Node:
    def __init__(self, value):
        self.value = value
        self.children = []
        
    def add_child(self, child_node):
        self.children.append(child_node)
        
class Tree:
    def __init__(self, root_value):
        self.root = Node(root_value)

    def find(self, value, node=None):
        if node is None:
            node = self.root
        if node.value == value:
            return node
        for child in node.children:
            result = self.find(value, child)
            if result is not None:
                return result
        return None

    def insert(self, parent_value, child_value):
        parent_node = self.find(parent_value)
        if parent_node is None:
            raise ValueError(f'Parent node with value {parent_value} not found.')
        child_node = Node(child_value)
        parent_node.add_child(child_node)

    def display(self, node=None, prefix="", is_last=True):
        if node is None:
            node = self.root
        print(prefix + ("└── " if is_last else "├── ") + str(node.value))
        prefix += "    " if is_last else "│   "
        child_count = len(node.children)
        for i, child in enumerate(node.children):
            is_last_child = i == (child_count - 1)
            self.display(child, prefix, is_last_child)

In [10]:
from collections import deque 
from collections import Counter

def branchmult(MAXG, m, S): #prints elements with multiplicity m in the num sgp tree starting at S
    tree = Tree(S.gens) #starts tree with generators of S as root 
    queue = deque([tree.root])
    g = S.genus
    numClst = []
    
    with open('/PATH_TO_FILE/MultChild.csv', 'w') as file:
        file.write('Genus,Generators,Number of Children, Kunz Coords\n')
        while queue and g <= MAXG:
            node = queue.popleft()
            current_gen = NumericalSemigroup(node.value)
            f = current_gen.genus
            C = current_gen.Children()
            numC = len(C)
            numClst.append(numC)
            
            # for current sgp print genus, min gens and # of children  
            # to a csv file 
            file.write(f'{g},"{node.value}",{numC},"{current_gen.Kunz_Coords()}"\n')

            for c in C: #for each num sgp c, in the list of children C of S
                tree.insert(node.value, c.gens)  
                # Insert the new semigroup into the tree
                if m in c.gens: #if c has multiplicity m
                    queue.append(Node(c.gens))  
                    # Append the new node to the queue 
           
            nextnode = queue[0]
            g = NumericalSemigroup(nextnode.value).genus
            
            if f < g: 
                print('genus', f, Counter(numClst))
                numClst = []

In [11]:
generators = [3,4,5]
semigroup = NumericalSemigroup(generators)
branchmult(30, 3, semigroup)

genus 2 Counter({3: 1})
genus 3 Counter({2: 1, 0: 1})
genus 4 Counter({2: 1, 0: 1})
genus 5 Counter({2: 1, 1: 1})
genus 6 Counter({2: 1, 1: 1, 0: 1})
genus 7 Counter({2: 1, 1: 1, 0: 1})
genus 8 Counter({1: 2, 2: 1})
genus 9 Counter({1: 2, 2: 1, 0: 1})
genus 10 Counter({1: 2, 2: 1, 0: 1})
genus 11 Counter({1: 3, 2: 1})
genus 12 Counter({1: 3, 2: 1, 0: 1})
genus 13 Counter({1: 3, 2: 1, 0: 1})
genus 14 Counter({1: 4, 2: 1})
genus 15 Counter({1: 4, 2: 1, 0: 1})
genus 16 Counter({1: 4, 2: 1, 0: 1})
genus 17 Counter({1: 5, 2: 1})
genus 18 Counter({1: 5, 2: 1, 0: 1})
genus 19 Counter({1: 5, 2: 1, 0: 1})
genus 20 Counter({1: 6, 2: 1})
genus 21 Counter({1: 6, 2: 1, 0: 1})
genus 22 Counter({1: 6, 2: 1, 0: 1})
genus 23 Counter({1: 7, 2: 1})
genus 24 Counter({1: 7, 2: 1, 0: 1})
genus 25 Counter({1: 7, 2: 1, 0: 1})
genus 26 Counter({1: 8, 2: 1})
genus 27 Counter({1: 8, 2: 1, 0: 1})
genus 28 Counter({1: 8, 2: 1, 0: 1})
genus 29 Counter({1: 9, 2: 1})
genus 30 Counter({1: 9, 2: 1, 0: 1})
